In [2]:
import sys
import subprocess

def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    except subprocess.CalledProcessError as e:
        print(f"Failed to install {package}: {e}")
        raise

def install_packages(packages):
    for package in packages:
        install_package(package)

try:
    # Check if running in Google Colab
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # Install the zenbase package if running in Google Colab
    # install_package('zenbase')
    # Install the zenbse package from a GitHub branch if running in Google Colab
    install_package('git+https://github.com/zenbase-ai/lib.git@main#egg=zenbase&subdirectory=py')

    # List of other packages to install in Google Colab
    additional_packages = [
        'python-dotenv',
        'parea-ai==0.2.164',
        'openai',
        'langchain',
        'langchain_openai'
    ]
    
    # Install additional packages
    install_packages(additional_packages)

# Now import the zenbase library
try:
    import zenbase
except ImportError as e:
    print("Failed to import zenbase: ", e)
    raise

In [3]:
from pathlib import Path
from dotenv import load_dotenv

# import os
#
# os.environ["OPENAI_API_KEY"] = "..."
# os.environ["PAREA_API_KEY"] = "..."

load_dotenv(Path("../../.env.test"), override=True)

True

In [4]:
import nest_asyncio

nest_asyncio.apply()

In [5]:
from openai import OpenAI
from parea import Parea, trace

parea = Parea()
openai = OpenAI()

parea.wrap_openai_client(openai)

In [6]:
from zenbase.core.managers import ZenbaseTracer

zenbase_manager = ZenbaseTracer()

In [20]:
from zenbase.types import LMRequest
import json
from zenbase.utils import expand_nested_json
from parea.schemas import Log, EvaluationResult


def score_answer_with_json(log: Log) -> EvaluationResult:
    print(expand_nested_json(log.output))
    if log.target:
        output = str(expand_nested_json(log.output)["answer"])
        target = log.target.split("#### ")[-1]
        return EvaluationResult("correctness", int(output == target))


@zenbase_manager.trace_function
@trace(eval_funcs=[score_answer_with_json])
def solver(request: LMRequest):
    print(request)
    messages = [  # noqa
        {
            "role": "system",
            "content": "You are an expert math solver. You have a question that you should answer. You have step by step actions that you should take to solve the problem. You have the operations that you should do to solve the problem. You should come just with the number for the answer, just the actual number like examples that you have. Follow the format of the examples as they have the final answer, you need to come up with the plan for solving them."  # noqa
            'return it with json like return it in the {"answer": " the answer "}',
        }
    ]

    for demo in request.zenbase.task_demos:
        messages += [
            {"role": "user", "content": f"Example Question: {str(demo.inputs)}"},
            {"role": "assistant", "content": f"Example Answer: {str(demo.outputs)}"},
        ]

    plan = planner_chain(request.inputs)
    the_plan = plan["plan"]
    the_operation = operation_finder(
        {
            "plan": the_plan,
            "question": request.inputs["question"],
        }
    )

    messages.append({"role": "user", "content": f"Question: {request.inputs['question']}"})
    messages.append({"role": "user", "content": f"Plan: {the_plan}"})
    messages.append(
        {"role": "user", "content": f"Mathematical Operation that needed: {the_operation['operation']}"}
    )
    messages.append(
        {
            "role": "user",
            "content": "Now come with the answer as number, just return the number, nothing else, just NUMBERS.",
        }
    )

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        response_format={"type": "json_object"},
    )

    print("Mathing...")
    answer = json.loads(response.choices[0].message.content)
    return {"answer": answer["answer"]}

@zenbase_manager.trace_function
@trace
def planner_chain(request: LMRequest):
    messages = [
        {
            "role": "system",
            "content": "You are an expert math solver. You have a question that you should create a step-by-step plan to solve it. Follow the format of the examples and return JSON object."  # noqa
            'return it in the {"plan": " the plan "}',
        }
    ]

    if request.zenbase.task_demos:
        for demo in request.zenbase.task_demos[:2]:
            messages += [
                {"role": "user", "content": str(demo.inputs)},
                {"role": "assistant", "content": str(demo.outputs)},
            ]
    if not request.inputs:
        messages.append({"role": "user", "content": "Question: What is 2 + 2?"})
    else:
        messages.append({"role": "user", "content": request.inputs["question"]})

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        response_format={"type": "json_object"},
    )

    print("Planning...")
    answer = json.loads(response.choices[0].message.content)
    return {"plan": " ".join(i for i in answer["plan"])}

@zenbase_manager.trace_function
@trace
def operation_finder(request: LMRequest):
    messages = [
        {
            "role": "system",
            "content": "You are an expert math solver. You have a plan for solving a problem that is step-by-step, you need to find the overall operation in the math to solve it. Just come up with math operation with simple math operations like sum, multiply, division and minus. Follow the format of the examples."  # noqa
            'return it with json like return it in the {"operation": " the operation "}',
        }
    ]

    if request.zenbase.task_demos:
        for demo in request.zenbase.task_demos[:2]:
            messages += [
                {"role": "user", "content": f"Input: {str(demo.inputs)}"},
                {"role": "assistant", "content": str(demo.outputs)},
            ]

    messages.append({"role": "user", "content": f"Question: {request.inputs['question']}"})
    messages.append({"role": "user", "content": f"Plan: {request.inputs['plan']}"})

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        response_format={"type": "json_object"},
    )

    print("Finding operation...")
    answer = json.loads(response.choices[0].message.content)
    return {"operation": answer["operation"]}


In [21]:
solver({"question": "What is 2+2?"})

LMRequest(zenbase=LMZenbase(task_demos=[], model_params={}), inputs={'question': 'What is 2+2?'}, id='request_2io00La9ysPEfWDhsv8TgPjJYcl')
Planning...
Finding operation...
Mathing...
{'answer': 4}


{'answer': '4'}

In [25]:
experiment_kwargs = dict(
    data= [
            {"inputs": json.loads(case.inputs["inputs"]), "target": case.target}
            for case in parea.get_collection("GSM8K_test_set_parea_dataset_2inu4fT1X5IZKj361H21BbD2Mc4").test_cases.values()],
    n_workers=1,
)


parea.experiment(
    name="wibbly-wobbly",
    func=solver,
    **experiment_kwargs
).run()

Run name set to: brute-deep, since a name was not provided.
LMRequest(zenbase=LMZenbase(task_demos=[], model_params={}), inputs={'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'}, id='request_2io0pRIsAn8YsSGjGTvRIlf33UL')


  0%|          | 0/5 [00:00<?, ?it/s]

Planning...
Finding operation...


 20%|██        | 1/5 [00:01<00:07,  1.99s/it]

Mathing...
{'answer': 72}
LMRequest(zenbase=LMZenbase(task_demos=[], model_params={}), inputs={'question': 'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?'}, id='request_2io0pcVweydButIArRgrUdxtoJH')
Planning...
Finding operation...


...

Finding operation...


100%|██████████| 5/5 [00:14<00:00,  3.00s/it]

Mathing...
{'answer': 624}



0it [00:04, ?it/s]


Experiment wibbly-wobbly Run brute-deep stats:
{
  "latency": "3.00",
  "input_tokens": "1287.40",
  "output_tokens": "137.80",
  "total_tokens": "1425.20",
  "cost": "0.00092",
  "correctness": "1.00"
}


View experiment & traces at: https://app.parea.ai/experiments/wibbly-wobbly/6aaced16-2627-4b55-beeb-a11a44ef52ba


In [26]:
solver({"question": "What is 2+2?"})

LMRequest(zenbase=LMZenbase(task_demos=[], model_params={}), inputs={'question': 'What is 2+2?'}, id='request_2io10wb1MkEf9JCBwHyVXbEBqXV')
Planning...
Finding operation...
Mathing...
{'answer': 4}


{'answer': '4'}

In [28]:
from zenbase.optim.metric.bootstrap_few_shot import BootstrapFewShot
from zenbase.optim.metric.labeled_few_shot import LabeledFewShot
from zenbase.adaptors.parea import ZenParea

zen_parea_helper = ZenParea(parea)

TRAIN_SET = "GSM8K_train_set_parea_dataset_2inu4fpM2Q5zIo0JDbtXZOeaG9Y"
TEST_SET = "GSM8K_test_set_parea_dataset_2inu4fT1X5IZKj361H21BbD2Mc4"
VALIDATION_SET = "GSM8K_validation_set_parea_dataset_2inu3r5jX6ruauOovt2rf5L5LHG"
SHOTS = 2
SAMPLES = 2

evaluator_kwargs = dict(
    p=parea,
    n_workers=1,
)

bootstrap_few_shot = BootstrapFewShot(
    shots=SHOTS,
    training_set=TRAIN_SET,
    test_set=TEST_SET,
    validation_set=VALIDATION_SET,
    evaluator_kwargs=evaluator_kwargs,
    zen_adaptor=zen_parea_helper,
)


teacher_lm = bootstrap_few_shot.perform(
    solver,
    samples=SAMPLES,
    rounds=1,
    trace_manager=zenbase_manager,
)


Run name set to: fluid-gonk, since a name was not provided.
LMRequest(zenbase=LMZenbase(task_demos=[], model_params={}), inputs={'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'}, id='request_2io1SxipDgfgVAgf96oWKA96vJM')


  0%|          | 0/5 [00:00<?, ?it/s]

Planning...
Finding operation...


 20%|██        | 1/5 [00:02<00:11,  2.81s/it]

Mathing...
{'answer': 72}
LMRequest(zenbase=LMZenbase(task_demos=[], model_params={}), inputs={'question': 'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?'}, id='request_2io1TS5vuygidtyYcXrB3SFNs9x')
Planning...
Finding operation...


...

Mathing...
{'answer': 42}
LMRequest(zenbase=LMZenbase(task_demos=[LMDemo(inputs={'question': 'Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?'}, outputs={'answer': '10'}, original_object=None), LMDemo(inputs={'question': 'Julie is reading a 120-page book. Yesterday, she was able to read 12 pages and today, she read twice as many pages as yesterday. If she wants to read half of the remaining pages tomorrow, how many pages should she read?'}, outputs={'answer': '42'}, original_object=None), LMDemo(inputs={'question': 'James writes a 3-page letter to 2 different friends twice a week.  How many pages does he write a year?'}, outputs={'answer': '624'}, original_object=None), LMDemo(inputs={'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many

100%|██████████| 5/5 [00:14<00:00,  2.98s/it]

Mathing...
{'answer': 624}



0it [00:04, ?it/s]


Experiment zenbase-adaptive-fresh-thinking Run adept-lift stats:
{
  "latency": "2.98",
  "input_tokens": "3034.80",
  "output_tokens": "141.00",
  "total_tokens": "3175.80",
  "cost": "0.00180",
  "correctness": "0.80"
}


View experiment & traces at: https://app.parea.ai/experiments/zenbase-adaptive-fresh-thinking/b01b1ea5-d629-4e40-a9be-daec2b0322ad
Num. experiments: 1


In [31]:
teacher_lm.best_function

functools.partial(<function BootstrapFewShot._create_optimized_function.<locals>.optimized_fn_base at 0x10794add0>, zenbase=LMZenbase(task_demos=[], model_params={}), optimized_args_in_fn={'planner_chain': {'args': {'zenbase': LMZenbase(task_demos=[LMDemo(inputs={'question': 'Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?'}, outputs={'plan': 'F i r s t ,   c a l c u l a t e   h o w   m u c h   m o n e y   B e t t y   h a s   s a v e d   s o   f a r   b y   a d d i n g   t h e   $ 1 5   f r o m   h e r   p a r e n t s   a n d   t h e   a m o u n t   f r o m   h e r   g r a n d p a r e n t s .   T h e n ,   c a l c u l a t e   h o w   m u c h   m o n e y   B e t t y   s t i l l   n e e d s   t o   s a v e   t o   b u y   t h e   w a l l e t   b y   s u b t r a c t i n g   t h e

In [34]:
# Now you can use your zenbase fn
teacher_lm.best_function({"question": "If I have 30% of shares, and Mo has 24.5% of shares, how many of our 10M shares are unassigned?"})

LMRequest(zenbase=LMZenbase(task_demos=[LMDemo(inputs={'question': 'Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?'}, outputs={'answer': '10'}, original_object=None), LMDemo(inputs={'question': 'Julie is reading a 120-page book. Yesterday, she was able to read 12 pages and today, she read twice as many pages as yesterday. If she wants to read half of the remaining pages tomorrow, how many pages should she read?'}, outputs={'answer': '42'}, original_object=None), LMDemo(inputs={'question': 'James writes a 3-page letter to 2 different friends twice a week.  How many pages does he write a year?'}, outputs={'answer': '624'}, original_object=None), LMDemo(inputs={'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many cl

{'answer': '4550000'}

In [36]:
zenbase_manager.all_traces = {}


In [37]:
# Now you can use your zenbase fn
teacher_lm.best_function({"question": "If I have 30% of shares, and Mo has 24.5% of shares, how many of our 10M shares are unassigned?"})

LMRequest(zenbase=LMZenbase(task_demos=[LMDemo(inputs={'question': 'Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?'}, outputs={'answer': '10'}, original_object=None), LMDemo(inputs={'question': 'Julie is reading a 120-page book. Yesterday, she was able to read 12 pages and today, she read twice as many pages as yesterday. If she wants to read half of the remaining pages tomorrow, how many pages should she read?'}, outputs={'answer': '42'}, original_object=None), LMDemo(inputs={'question': 'James writes a 3-page letter to 2 different friends twice a week.  How many pages does he write a year?'}, outputs={'answer': '624'}, original_object=None), LMDemo(inputs={'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many cl

{'answer': '4500000'}

In [38]:
function_traces = [v for k, v in zenbase_manager.all_traces.items()][0]["optimized"]

In [41]:
from pprint import pprint

pprint(function_traces["solver"]["args"]["request"].zenbase.task_demos)


[LMDemo(inputs={'question': 'Betty is saving money for a new wallet which '
                            'costs $100. Betty has only half of the money she '
                            'needs. Her parents decided to give her $15 for '
                            'that purpose, and her grandparents twice as much '
                            'as her parents. How much more money does Betty '
                            'need to buy the wallet?'},
        outputs={'answer': '10'},
        original_object=None),
 LMDemo(inputs={'question': 'Julie is reading a 120-page book. Yesterday, she '
                            'was able to read 12 pages and today, she read '
                            'twice as many pages as yesterday. If she wants to '
                            'read half of the remaining pages tomorrow, how '
                            'many pages should she read?'},
        outputs={'answer': '42'},
        original_object=None),
 LMDemo(inputs={'question': 'James writes a 3-

In [42]:
pprint(function_traces["operation_finder"]["args"]["request"].zenbase.task_demos)


[LMDemo(inputs={'plan': 'F i r s t ,   c a l c u l a t e   h o w   m u c h   m '
                        'o n e y   B e t t y   h a s   s a v e d   s o   f a '
                        'r   b y   a d d i n g   t h e   $ 1 5   f r o m   h e '
                        'r   p a r e n t s   a n d   t h e   a m o u n t   f r '
                        'o m   h e r   g r a n d p a r e n t s .   T h e n ,   '
                        'c a l c u l a t e   h o w   m u c h   m o n e y   B e '
                        't t y   s t i l l   n e e d s   t o   s a v e   t o   '
                        'b u y   t h e   w a l l e t   b y   s u b t r a c t i '
                        'n g   t h e   t o t a l   a m o u n t   s h e   h a '
                        's   s a v e d   f r o m   t h e   c o s t   o f   t h '
                        'e   w a l l e t .',
                'question': 'Betty is saving money for a new wallet which '
                            'costs $100. Betty has only half of the money

In [45]:
pprint(function_traces["planner_chain"]["args"]["request"].zenbase.task_demos)


[LMDemo(inputs={'question': 'Betty is saving money for a new wallet which '
                            'costs $100. Betty has only half of the money she '
                            'needs. Her parents decided to give her $15 for '
                            'that purpose, and her grandparents twice as much '
                            'as her parents. How much more money does Betty '
                            'need to buy the wallet?'},
        outputs={'plan': 'F i r s t ,   c a l c u l a t e   h o w   m u c h   '
                         'm o n e y   B e t t y   h a s   s a v e d   s o   f '
                         'a r   b y   a d d i n g   t h e   $ 1 5   f r o m   '
                         'h e r   p a r e n t s   a n d   t h e   a m o u n '
                         't   f r o m   h e r   g r a n d p a r e n t s .   T '
                         'h e n ,   c a l c u l a t e   h o w   m u c h   m o '
                         'n e y   B e t t y   s t i l l   n e e d s   t o 